In [1]:
%matplotlib inline

In [2]:
import unittest
from unis.models import *
from unis.runtime import Runtime
import networkx as nx
import itertools

In [60]:
def test(case=None):
    import sys

    unis._runtime_cache = {}
    print("Cleared unis Runtime cache")

    if case is None:
        module = sys.modules[__name__]
        suite = unittest.TestLoader().loadTestsFromModule(module)
    else:
        suite = unittest.TestLoader().loadTestsFromTestCase(case)
    unittest.TextTestRunner().run(suite)

In [61]:
class FlangeTree(object):
    def __call__(self): 
        raise RuntimeError("Must override the call method")

class ActionFailureError(RuntimeError): 
    "Inidicates rule action was taken BUT the test still fails"
    pass

class NoValidChoice(RuntimeError):
    "Indicates that a search returned zero valid options."
    pass

        
def update(graph, values): pass

In [62]:
class rule(FlangeTree): 
    """
    A test/action pair. Executes the action if the rule fails.
    Returns the result of action (if taken) or the original arguments (if no action taken).
    
    The result of the action must be a list of things of the same type as the args.
    This makes the test/action/retest work properly.
    """

    def __init__(self, test, action):
        self.test = test
        self.action = action
        
    def __call__(self, *args):
        rslt=args
        if not self.test(*args): 
            rslt = self.action(*args)
            if not self.test(*rslt):
                raise ActionFailureError()
            return rslt   
        return rslt
            
class Test_rule(unittest.TestCase):
    def test_action_pass(self):
        c = rule(lambda x: 4<x[0], lambda x: [[6]])
        self.assertEqual(c([3]), [[6]])

    def test_simple_pass(self):
        c = rule(lambda x: 4>3, lambda x: [[3]])
        self.assertEqual(c(5), (5,))
        
    def test_double_fail(self):
        c = rule(lambda *x: 4<3, lambda *x: [4])
        self.assertRaises(ActionFailureError, c)
        

In [63]:
class switch(FlangeTree):
    "A rule that always tests to True."
    def default(action): return rule(lambda *x: True, action)

    "Tests rules in order. Does the action associated with the first item test that passes."
    def __init__(self, *rules):
        self.rules = rules

    def __call__(self):
        for rule in self.rules:
            if rule.test():
                return rule.action()
        raise NoValidChoice("No action take in switch")

class Test_switch(unittest.TestCase):
    FALSE = lambda *x: False
    TRUE = lambda *x: True
    
    def test_last(self):
        s = switch(rule(self.FALSE, lambda *x: None), 
                   rule(self.FALSE, lambda *x: None), 
                   rule(self.FALSE, lambda *x: None), 
                   switch.default(lambda *x: 10))
        self.assertEqual(s(), 10)
        
    def test_intermediate(self):
        s = switch(rule(self.FALSE, lambda *x: None), 
                   rule(self.TRUE, lambda *x: 6), 
                   rule(self.FALSE, lambda *x: None), 
                   switch.default(lambda *x: 10))
        self.assertEqual(s(), 6)
        
    def test_default_rule(self):
        r = switch.default(lambda *x: 3)
        self.assertEqual(r.test(), True)
        
    def test_no_valid(self):
        s = switch(rule(self.FALSE, lambda *x: None), 
                   rule(self.FALSE, lambda *x: None), 
                   rule(self.FALSE, lambda *x: None))
        self.assertRaises(NoValidChoice, s)

In [89]:
class unis(FlangeTree):
    "Retrieves a graph from a UNIS server."
    default_unis = "http://192.168.100.200:8888"
    _runtime_cache = {}
    
    def __init__(self, ref="*", source=default_unis):
        self.source = source
        self.ref = ref

    def __call__(self):
        rt = self.cached_connection(self.source)
        
        #TODO: This is a bare-bones buildling of the graph model
        if self.ref == "*": topology = rt
        else: topology= list(rt.topologies.where({"id": self.ref}))[0]

        g = nx.Graph() #TODO: Digraph?
        for port in topology.ports:
            g.add_node(port["id"], **port)
        for link in topology.links:
            g.add_edge(link.endpoints[0], link.endpoints[1], object=link, **link)
        
        return g
    
    @classmethod
    def cached_connection(cls, source):
        rt = cls._runtime_cache.get(source, Runtime(source))
        cls._runtime_cache[source] = rt
        return rt

class Test_unis(unittest.TestCase):
    explicit_host = "http://192.168.100.200:8888"
    
    def setUpClass():
        unis._runtime_cache = {}
        print("Cleared unis Runtime cache")

    def test_implict_host(self):
        b = unis()
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)

    def test_all_implicit(self):
        b = unis(source=self.explicit_host)
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)
        
    def test_all_explicit(self):
        b = unis("*", self.explicit_host)
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)
        
    def test_named(self):
        b = unis("test", self.explicit_host)
        g = b()
        self.assertEqual(len(g.nodes()), 2)
        self.assertEqual(len(g.edges()), 1)

In [121]:
class graph(FlangeTree):
    linear = {"nodes": ["port1","port2","port3","port4"],
              "edges": [("port1", "port2"), ("port2", "port3"),("port3", "port4")]}

    def __init__(self, topology="linear", nodes=None, edges=None):
        g = nx.Graph()

        if nodes or edges:
            topology = {"nodes": nodes if nodes else [],
                        "edges": edges if edges else []}
        else:
            topology = graph.__getattribute__(graph, topology)
        
        
        for port in topology["nodes"]:
            g.add_node(port, id=port)
            
        for link in topology["edges"]:
            g.add_edge(link[0], link[1])
            
        self.graph = g
    
    def __call__(self):
        return self.graph
    
class Test_graph(unittest.TestCase):
    def test_linear(self):
        g = graph()()
        self.assertEqual(len(g.nodes()), 4)
        self.assertEqual(len(g.edges()), 3)

In [127]:
class GroupCondition(FlangeTree):
    def __init__(self, predicate, selector, graph): 
        self.predicate = predicate
        self.selector = selector
        self.graph = graph
    
    def __call__(self, *args):
        g = self.graph()
        raw_items = self.selector(g)
        passing_items = [x for x in raw_items if self.predicate(x, g)]
        return self._test(passing_items, raw_items)
    
    def _test(self, passing_items, raw_items): 
        raise Error("Not implemented")
    
class exists(GroupCondition):
    def _test(self,  passing_items, raw_items): 
        return len(passing_items) > 0
    
class exactlyOne(GroupCondition):
    def _test(self, passing_items, raw_items): 
        return len(passing_items) == 1
    
class all(GroupCondition):
    def _test(self, passing_items, raw_items): 
        return len(passing_items) == len(raw_items)

class most(GroupCondition):
    def _test(self, passing_items, raw_items): 
        return len(passing_items) >= len(raw_items)//2
    
class TestGroupConditions(unittest.TestCase):
    def test_exists_pass(self):
        t = exists(lambda n, g: g.degree(n) > 0, 
                   lambda g: g.nodes(), 
                   graph())
        self.assertTrue(t())
        
    def test_exists_fail(self):
        t = exists(lambda n, g: g.degree(n) == 0, 
                   lambda g: g.nodes(), 
                   graph())
        self.assertFalse(t())

    def test_exactlyone_pass(self):
        t = exists(lambda n, g: g.node[n]["id"] == "port1", 
                   lambda g: g.nodes(), 
                   graph())
        self.assertTrue(t())
        
    def test_exactlyOne_fail(self):
        t = exactlyOne(lambda n, g: g.degree(n) == 0, 
                   lambda g: g.nodes(), 
                   graph())
        self.assertFalse(t())
        
        t = exactlyOne(lambda n, g: g.degree(n) > 0, 
                   lambda g: g.nodes(), 
                   graph())
        self.assertFalse(t())
            
    def test_all_pass(self):
        t = all(lambda n, g: g.degree(n) > 0, 
                   lambda g: g.nodes(), 
                   graph())
        self.assertTrue(t())

    def test_all_fail(self):
        t = all(lambda n, g: g.node[n]["id"] == "port1", 
                   lambda g: g.nodes(), 
                   graph())
        self.assertFalse(t())

    def test_most_pass(self): 
        t = most(lambda n, g: int(g.node[n]["id"][-1]) < 3,
                 lambda g: g.nodes(), 
                 graph())
        self.assertTrue(t())
        
    def test_most_fail(self):
        t = most(lambda n, g: g.node[n]["id"] == "port3",
                 lambda g: g.nodes(), 
                 graph())
        self.assertFalse(t())



In [198]:
class inside(FlangeTree):
    def __init__(self, selector):
        self.selector=selector
                
    def __call__(self, graph):
        """Given a graph and a selector, returns a list of links that make up the min cut"""
        selector = lambda x: self.selector(x, graph)
        nodes = list(filter(selector, graph.nodes())) ## TODO -- More complex query here
        synth = graph.subgraph(nodes) ## Get the subgraph indicated by the selector
        nx.set_edge_attributes(synth, 'capacity', 1)

        #outbound = graph.out_edges_iter(nodes)  -- Switch unis to directed graph, then use
        #inbound = graph.in_edges_iter(nodes)
        outbound = graph.edges_iter(nodes)
        inbound = graph.edges_iter(nodes)


        src = "##SOURCE##"
        synth.add_node(src)
        for edge in inbound:
            synth.add_edge(src, edge[1], capacity=10)

        sink = "##SINK##"
        synth.add_node(sink)
        for edge in outbound:
            synth.add_edge(edge[0], sink, capacity=10)

        cut_value, partition = nx.minimum_cut(synth, s=src, t=sink, capacity='capacity')
        reachable, non_reachable = partition

        cutset = set()
        for u, nbrs in ((n, synth[n]) for n in reachable):
            cutset.update((u, v) for v in nbrs if v in non_reachable)
            cut_value == sum(synth.edge[u][v]['capacity'] for (u, v) in cutset)

        return cutset

class Test_inside(unittest.TestCase):
    def test(self):
        i = inside(lambda x,g: int(g.node[x]["id"][-1]) < 3)(graph()())
        r = {(e[0], e[1]) for e in i}
        self.assertEqual(r, {("port1", "port2")})

In [205]:
class on(FlangeTree):
    def __init__(self, selector):
        self.selector = selector
        
    def __call__(self, graph):
        selector = lambda x: self.selector(x, graph)
        nodes = list(filter(selector, graph.nodes()))
        synth = graph.subgraph(nodes)
        return synth
    
class Test_on(unittest.TestCase):
    def test(self):
        o = on(lambda x,g: int(g.node[x]["id"][-1]) < 3)(graph()())
        self.assertEqual(["port1", "port2"], o.nodes())
        self.assertEqual(1, len(o.edges()))
        

In [210]:
class place(FlangeTree):
    def __init__(self, mod, at, graph):
        self.mod = mod
        self.at = at
        self.graph = graph
    
    def __call__(self, *args):
        g = self.graph()
        position = self.at(g)
        g2 = self.mod(position, g)
        return g2

class Test_place(unittest.TestCase):
    def test_placement(self):
        p = place(lambda positions, g: {g.node[n]["id"]: "modified" for n in positions},
                  on(lambda x,g: int(g.node[x]["id"][-1]) < 3),
                  graph())
        self.assertEqual(p(), {"port1": "modified", "port2": "modified"})


In [141]:
class around(FlangeTree):
    def __init__(self, selector):
        self.selector = selector
        
    def __call__(self, graph):
        """Given a graph and a selector, returns a list of links that go in or out of the selected nodes"""
        selector = lambda x: self.selector(x,graph)
        nodes = list(filter(selector, graph.nodes())) ## TODO -- More complex query here
        outbound = graph.edges_iter(nodes)
        inbound = graph.edges_iter(nodes)
        return set(itertools.chain(outbound, inbound))

class Test_around(unittest.TestCase):
    def test(self):
        i = around(lambda x, g: g.node[x]["id"] == "port1")(graph()())
        r = {(e[0], e[1]) for e in i}
        self.assertEqual(r, {("port1", "port2")})

In [176]:
class near(FlangeTree):
    def __init__(self, criteria, selector):
        self.criteria = criteria
        self.selector = selector
        
    def __call__(self, graph):
        """Find a node that is topologically 'near' the selection and passes the criteria.

        * Nearness is topological nearness (based on BFS)
        * Criteria is a function that determines if a node passes the selection criteria

        TODO: Custom/pluggable nearness functions (such as BFS or custom weighting functions)
        """

        selector = lambda x: self.selector(x, graph)
        criteria = lambda x: self.criteria(x, graph)
        sources = list(filter(selector, graph.nodes()))

        def path_weight(path):
            # TODO: look up  weights in the graph 
            return len(path)

        all_paths = {s:nx.shortest_path(graph, source=s) for s in sources}
        paths = [steps for (s, paths) in all_paths.items() 
                       for (t, steps) in paths.items()]

        distances = [(path[-1], path_weight(path)) for path in paths if len(path) > 1]
        valid = list(filter(criteria, distances))
        if len(valid) == 0: raise NoValidChoice()

        preferred = min(valid, key=lambda e: e[1])

        return preferred[0]        

class Test_near(unittest.TestCase):
    def test(self): 
        g = graph()()
        target = g["port2"]
        n = near(lambda x, g: True, lambda x,g: g[x] == target)(g)
        self.assertEqual(n, "port1")

In [71]:
def between(criteria, src_selector, dst_selector, graph): 
    """Finds a path bewteen the source and target selector where each hop
       statisfies the given criteria.

    TODO: handle multple sources or targets...
    TODO: Modify so it finds a set of paths that collectively satisfy the 
            criteria instead of just one path that always does.  That way
            you can build a flow from A to B where no individual link is 
            sufficient but combinations of links are.
    """
    
    try:
        s = next(filter(src_selector, g.nodes()))
        t = next(filter(dst_selector, g.nodes()))
    except StopIteration:
        raise NoValidChoice("Source or target set empty")
        
    allpaths = nx.all_simple_paths(graph, s, t)
    return allpaths
    
    
    nx.shortest_path(s, t)
    
    # Get all paths froms src to target
    # For each path, find a spot that matches criteria
    # Do something "smart" so you use a few spots as possible (so if paths intersect and the intersection matches, use it)
    # exclude SRC and target
    pass

In [212]:
test()

.....................EEEE

Cleared unis Runtime cache
Cleared unis Runtime cache



ERROR: test_all_explicit (__main__.Test_unis)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-89-89bcdc13f33c>", line 52, in test_all_explicit
    g = b()
  File "<ipython-input-89-89bcdc13f33c>", line 19, in __call__
    g.add_node(port["id"], **port)
TypeError: 'Port' object is not subscriptable

ERROR: test_all_implicit (__main__.Test_unis)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-89-89bcdc13f33c>", line 46, in test_all_implicit
    g = b()
  File "<ipython-input-89-89bcdc13f33c>", line 19, in __call__
    g.add_node(port["id"], **port)
TypeError: 'Port' object is not subscriptable

ERROR: test_implict_host (__main__.Test_unis)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-89-89bcdc13f33c>", line 40, in test_implict_host
    g = b()
 

In [28]:
def populate(url):
  with Runtime(url) as rt:
    node1 = Node({"id": "node1"})
    node2 = Node({"id": "node2"})
    node3 = Node({"id": "node3"})
    node4 = Node({"id": "node4"})
    port1 = Port({"id": "port1"})
    port2 = Port({"id": "port2"})
    port3 = Port({"id": "port3"})
    port4 = Port({"id": "port4"})
    
    node1.ports.append(port1)
    node2.ports.append(port2)
    node2.ports.append(port3)
    node2.ports.append(port4)
    link1 = Link({"id": "link1-2", "directed": False, "endpoints": [port1, port2]})
    link2 = Link({"id": "link2-3", "directed": False, "endpoints": [port2, port3]})
    link3 = Link({"id": "link3-4", "directed": False, "endpoints": [port3, port4]})
    topology = Topology({"id": "test", 
                         "ports" : [port1, port2, port3, port4],
                         "nodes": [node1, node2, node3, node4], 
                         "links" : [link1, link2, link3]})
    rt.insert(port1, commit=True)
    rt.insert(port2, commit=True)
    rt.insert(port3, commit=True)
    rt.insert(port4, commit=True)
    rt.insert(node1, commit=True)
    rt.insert(node2, commit=True)
    rt.insert(node3, commit=True)
    rt.insert(node4, commit=True)
    rt.insert(link1, commit=True)
    rt.insert(link2, commit=True)
    rt.insert(link3, commit=True)
    rt.insert(topology, commit=True)

In [29]:
def populate(url):
  with Runtime(url) as rt:
    node1 = Node({"id": "node1"})
    port1 = Port({"id": "port1"})
    
    node1.ports.append(port1)
    rt.insert(port1, commit=True)
    rt.insert(node1, commit=True)

In [31]:
populate("http://192.168.100.200:8888")

ValueError: Item does not exist in collection